In [1]:
!pip install git+https://github.com/huggingface/transformers accelerate flash_attn
!pip install qwen_vl_utils av pyav

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-_r3vpg7x
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-_r3vpg7x
  Resolved https://github.com/huggingface/transformers to commit f2c388e3f946862f657acc1e21b272ec946fc66c
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 32.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 54.0 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.46.0.dev0-py3-none-any.whl size=9919788 sha256=d8cd20bf32bcaf15f615373320a15f16ed72129a76d88960522d1c872bd0dcff
  Stored in directory: /tmp/pip-ephem-wheel-cache-smbir7kf/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
  Created wheel for flash_attn: filename=flash_attn-2.


git+https://github.com/huggingface/transformers: This part installs the Hugging Face transformers library directly from the official GitHub repository. This is useful for getting the latest version (even if it's not on PyPI yet).

flash_attn: FlashAttention library is high-performance implementation of attention mechanisms in Transformer models. FlashAttention is designed to be faster and more memory-efficient than standard attention implementations.

qwen_vl_utils: This package provides utilities for Qwen-VL, which is a vision-language model from Alibaba. It is useful for handling multimodal tasks (e.g., image and text processing together).

av: The av library provides bindings to FFmpeg, a multimedia framework. It is used for processing multimedia content like video and audio, so if you're working with tasks involving video (such as in multimodal models), this package could be necessary.

In [2]:
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info
import av
import torch

AutoProcessor: Similar to AutoTokenizer, this is a utility class that automatically selects the appropriate processor based on the model. Processors are used in multimodal models to handle complex data (e.g., images, text, or a combination). In the case of vision-language models, a processor typically handles the pre-processing of both image and text inputs to make them ready for the model.

process_vision_info: This function is probably responsible for transforming raw visual data (e.g., images) into a suitable format, such as resizing, normalizing, or converting the image into tensors that the Qwen-VL model can understand.

In [3]:
model_name = "Qwen/Qwen2-VL-2B-Instruct"

In [21]:
# model = Qwen2VLForConditionalGeneration.from_pretrained(
#     model_name, torch_dtype=torch.bfloat16, attn_implementation="flash_attention_2", device_map="auto"
# )

model = Qwen2VLForConditionalGeneration.from_pretrained(
    model_name, torch_dtype=torch.bfloat16, attn_implementation="eager", device_map="auto"
)



Unrecognized keys in `rope_scaling` for 'rope_type'='default': {'mrope_section'}


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
processor = AutoProcessor.from_pretrained(model_name)

preprocessor_config.json:   0%|          | 0.00/347 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

In [11]:
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "video",
                "video": "/content/sample_data/video.mp4",
                "max_pixels": 360 * 420,
                "fps": 1.0,
            },
            {"type": "text", "text": "what happened in the video?"},
        ],
    }
]

In [7]:
messages

[{'role': 'user',
  'content': [{'type': 'video',
    'video': '/content/sample_data/video.mp4',
    'max_pixels': 151200,
    'fps': 1.0},
   {'type': 'text', 'text': 'what happened in the video?'}]}]

In [8]:
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)

In [9]:
!apt-get install ffmpeg -y

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [15]:
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)

In [17]:
inputs = inputs.to("cuda")

In [22]:
generated_ids = model.generate(**inputs, max_new_tokens=512)

In [23]:
# generated_ids = model.generate(**inputs, max_length=inputs['input_ids'].shape[1] + 512)


In [24]:
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]

In [25]:
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)

In [26]:
print(output_text)

['The video shows a large parking lot with several tires lined up in the foreground. There are also several cars parked in the lot. The sky is blue with some clouds, and the overall scene appears to be a parking lot or a similar outdoor area.']
